In [1]:
import json
import random
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from tenacity import retry, stop_after_attempt, wait_fixed
import schedule
import time
import pickle
import os
import logging
import datetime

# Setup logging
logging.basicConfig(filename='automation.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Load the JSON file
def load_comment_parts(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

class LinkedInAutomation:
    def __init__(self, driver_path, url):
        options = webdriver.ChromeOptions()
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        self.service = Service(driver_path)
        self.driver = webdriver.Chrome(service=self.service, options=options)
        self.url = url

    def load_cookies(self, cookies_file):
        self.driver.get(self.url)
        self.driver.maximize_window()
        if os.path.exists(cookies_file):
            cookies = pickle.load(open(cookies_file, 'rb'))
            for cookie in cookies:
                self.driver.add_cookie(cookie)
            self.driver.refresh()
            time.sleep(10)

    def generate_comment(self, comment_parts):
        greeting = random.choice(comment_parts['greetings'])
        topic = random.choice(comment_parts['topics'])
        detail = random.choice(comment_parts['details'])
        specific_detail = random.choice(comment_parts['specific_details'])
        return f"{greeting} {topic}! {detail} {specific_detail}."

    @retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
    def search_and_comment(self, search_url, comment_parts):
        self.driver.get(search_url)
        
        all_buttons = self.driver.find_elements(By.TAG_NAME, "button")
        logger.info(f'Found {len(all_buttons)} buttons')
        comment_buttons = [btn for btn in all_buttons if btn.get_attribute('textContent').strip() == "Comment"]
        logger.info(f'Found {len(comment_buttons)} comment buttons')

        time.sleep(5)
        if comment_buttons and comment_buttons[0].is_enabled():
            comment_buttons[0].click()
            
            # Wait for the comment box to be visible
            try:
                commentbox = WebDriverWait(self.driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, "//div[@aria-Label='Text editor for creating content']"))
                )
                logger.info('Comment box found and is visible')
                
                # Generate a comment based on the comment parts
                comment_text = self.generate_comment(comment_parts)
                commentbox.send_keys(comment_text)
                time.sleep(10)

                post_buttons = self.driver.find_elements(By.TAG_NAME, 'button')
                post = [btn for btn in post_buttons if btn.get_attribute('textContent').strip() == "Comment"]
                logger.info(f'Found {len(post)} post buttons')
                if post and post[1].is_enabled():
                    post[1].click()
                    logger.info('Comment posted successfully')
                else:
                    logger.error('Post button not found or not enabled')

            except Exception as e:
                logger.error(f'Error interacting with the comment box: {e}')

    def close(self):
        time.sleep(10)
        self.driver.quit()

def job():
    driver_path = r"C:\Users\user\Downloads\chromedriver-win32\chromedriver-win32\chromedriver.exe"
    login_url = "https://www.linkedin.com/login"
    cookies_file = 'linkedin-cookies.pkl'

    automation = LinkedInAutomation(driver_path, login_url)
    automation.load_cookies(cookies_file)

    comment_parts = load_comment_parts('comments.json')
    
    search_urls = [
        'https://www.linkedin.com/search/results/content/?datePosted=%22past-24h%22&keywords=machine%20learning&origin=FACETED_SEARCH&sid=W(s',
        'https://www.linkedin.com/search/results/CONTENT/?keywords=data%20science&origin=SWITCH_SEARCH_VERTICAL&sid=q)D'
    ]

    for url in search_urls:
        automation.search_and_comment(url, comment_parts)

    logger.info('Automation Complete')
    automation.close()

if __name__ == "__main__":
    # Schedule the job to run a minute from now for testing purposes
    now = datetime.datetime.now()
    run_time = (now + datetime.timedelta(minutes=1)).strftime("%H:%M")
    schedule.every().day.at(run_time).do(job)

    # Run the schedule for a short duration to test
    end_time = datetime.datetime.now() + datetime.timedelta(minutes=2)  # Run for 2 minutes
    while datetime.datetime.now() < end_time:
        schedule.run_pending()
        time.sleep(1)

In [ ]:
#Adujust Schedule as needed
if __name__ == "__main__": 
    # Schedule the job every day at a specific time 
    schedule.every().day.at("09:00").do(job) 
    
    while True: 
        schedule.run_pending() 
        time.sleep(1)